#                                  Selenium Assignment3

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import selenium
from urllib.parse import urlparse, parse_qs
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import re

# Q1.  Write a python program which searches all the product under a particular product from www.amazon.in. 


In [202]:
driver2 = webdriver.Chrome(r"chromedriver.exe")
driver2.get('https://www.amazon.in/')

In [203]:
user = input('please enter product here: ')


please enter product here: headphones


In [206]:
search_bar = driver2.find_element(By.XPATH,'//*[@id="twotabsearchtextbox"]')    # Finding the search bar using it's xpath
search_bar.send_keys(user)       # Inputing keyword to search 
search_button = driver2.find_element(By.XPATH,'//*[@id="nav-search-submit-button"]')    # Finding the xpath of search button
search_button.click()        # Clicking the search button

In [207]:
productName=[]

In [209]:
#scraping the Product_Name 
PName=driver2.find_elements(By.XPATH,"//span[@class='a-size-medium a-color-base a-text-normal']")
for i in PName:
    if i.text is None :
        productName.append("--") 
    else:
        productName.append(i.text)
# print(len(productName),productName)

# Q2. In the above question, now scrape the following details of each product listed in first 3 pages.

In [217]:
start = 0
end = 3
urls = []
for page in range(start,end+1):
    try:
        page_urls = driver2.find_elements(By.XPATH,'//a[@class="a-link-normal s-no-outline"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')     # Scraping the url from webelement
            if url[0:4]=='http':                # Checking if the scraped data is a valid url or not
                urls.append(url)                # Appending the url to urls list
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
        nxt_button = driver2.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')      # Locating the next_button which is active
        if nxt_button.text == 'Next':                                            # Checking if the button located is next button
            nxt_button.click()                                                    # Clicking the next button
            time.sleep(5)                                                         # time delay of 5 seconds
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        elif driver2.find_element(By.XPATH,'//li[@class="a-disabled a-last"]/a').text == 'Next':    
            print("No new pages exist. Breaking the loop")  # Printing message and breakinf loop if we have reached the last page
            break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        # Extracting the url of next page
        driver2.get(next_page)                             # ReLoading the next page

Product urls of page 1 has been scraped.
Product urls of page 2 has been scraped.
Product urls of page 3 has been scraped.
Product urls of page 4 has been scraped.


In [224]:
prod_dict = {}
prod_dict['Brand']=[][:30]
prod_dict['Name']=[][:30]
prod_dict['Rating']=[][:30]
prod_dict['No. of ratings']=[][:30]
prod_dict['Price']=[][:30]
prod_dict['Expected Delivery']=[][:30] 
prod_dict['Availability']=[][:30]
prod_dict['Other Details']=[][:30]
prod_dict['URL']=[][:30]

In [226]:
for url in urls[:4]:
    driver2.get(url)                                                        # Loading the webpage by url
    print("Scraping URL = ", url)
    #time.sleep(2)
    
    try:
        brand = driver2.find_element(By.XPATH,'//a[@id="bylineInfo"]')      # Extracting Brand from xpath
        prod_dict['Brand'].append(brand.text)
    except NoSuchElementException:
        prod_dict['Brand'].append('-')
    
    try:
        name = driver2.find_element(By.XPATH,'//h1[@id="title"]/span')      # Extracting Name from xpath
        prod_dict['Name'].append(name.text)
    except NoSuchElementException:
        prod_dict['Name'].append('-')
    
    try:
        rating = driver2.find_element(By.XPATH,'//span[@id="acrPopover"]')  # Extracting Ratings from xpath
        prod_dict['Rating'].append(rating.get_attribute("title"))
    except NoSuchElementException:
        prod_dict['Rating'].append('-')
    
    try:
        n_rating = driver2.find_element(By.XPATH,'//a[@id="acrCustomerReviewLink"]/span')     # Extracting no. of Ratings from xpath
        prod_dict['No. of ratings'].append(n_rating.text)
    except NoSuchElementException:
        prod_dict['No. of ratings'].append('-')
    
    try:
        price = driver2.find_element(By.XPATH,'//span[@class="a-price-whole"]')            # Extracting Price from xpath
        prod_dict['Price'].append(price.text)
    except NoSuchElementException:
        prod_dict['Price'].append('-')

    try:
        delivry = driver2.find_element(By.XPATH,'//span[@class="a-text-bold"]')         # Extracting Expected Delivery from xpath
        prod_dict['Expected Delivery'].append(delivry.text)
    except NoSuchElementException:
        prod_dict['Expected Delivery'].append('-')
    
    try:
        avl = driver2.find_element(By.XPATH,'//div[@id="availability"]/span')                # Extracting Availability from xpath
        prod_dict['Availability'].append(avl.text)
    except NoSuchElementException:
        prod_dict['Availability'].append('-')
    
    try:                                                                                    # Extracting Other Details from xpath
        dtls = driver2.find_element(By.XPATH,'//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
        prod_dict['Other Details'].append('  ||  '.join(dtls.text.split('\n')))
    except NoSuchElementException:
        prod_dict['Other Details'].append('-')
    
    prod_dict['URL'].append(url)                                                            # Saving url
    time.sleep(2)

Scraping URL =  https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo4Njg5NzcxMTU2MzMxMzkxOjE2ODYzOTc3NDI6c3BfYXRmOjIwMDY4Nzk4MTY5NDAxOjowOjo&url=%2FHiFiMAN-HE400SE-Wired-Headphone-Silver%2Fdp%2FB08Z2SK5C4%2Fref%3Dsr_1_1_sspa%3Fkeywords%3Dheadphones%26qid%3D1686397742%26sr%3D8-1-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1
Scraping URL =  https://www.amazon.in/sspa/click?ie=UTF8&spc=MTo4Njg5NzcxMTU2MzMxMzkxOjE2ODYzOTc3NDI6c3BfYXRmOjIwMTQ4NzkxOTM0OTk4OjowOjo&url=%2FRD-Bluetooth-Headphone-Four-Microphone-Surrounding%2Fdp%2FB09SFX5Z63%2Fref%3Dsr_1_2_sspa%3Fkeywords%3Dheadphones%26qid%3D1686397742%26sr%3D8-2-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1
Scraping URL =  https://www.amazon.in/Rockerz-450-Wireless-Bluetooth-Headphone/dp/B07PR1CL3S/ref=sr_1_3?keywords=headphones&qid=1686397742&sr=8-3
Scraping URL =  https://www.amazon.in/boAt-Rockerz-550-Headphone-Aesthetics/dp/B08R7M1JM5/ref=sr_1_4?keywords=headphones&qid=1686397742&sr=8-4


In [227]:
df1 = pd.DataFrame.from_dict(prod_dict)
df1

,Brand,Name,Rating,No. of ratings,Price,Expected Delivery,Availability,Other Details,URL
0,Brand: HiFiMAN,HiFiMAN HE400SE Stealth Magnets Version Over-E...,4.4 out of 5 stars,624 ratings,"10,999","Wednesday, 14 June",In stock,What is a planar headphone? What are its benef...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Visit the RD Store,RD BH-205 Pure Bass Bluetooth 5.0 Wireless Ove...,3.3 out of 5 stars,29 ratings,699,"Monday, 12 June",In stock,Bluetooth Version 5.0:- With latest Bluetooth ...,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Visit the boAt Store,boAt Rockerz 450 Bluetooth On Ear Headphones w...,4.1 out of 5 stars,"114,768 ratings","1,499","Monday, 12 June",In stock,Playback- It provides a massive battery backup...,https://www.amazon.in/Rockerz-450-Wireless-Blu...
3,Visit the boAt Store,boAt Rockerz 550 Bluetooth Wireless Over Ear H...,4.1 out of 5 stars,"58,998 ratings","1,999","Wednesday, 14 June",In stock,Playback- The mighty 500mAh battery capacity o...,https://www.amazon.in/boAt-Rockerz-550-Headpho...


In [228]:
driver2.close()

# Q3: Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’. 

In [8]:

import urllib

# Function to scrape images for a given keyword
def scrape_images(keyword, num_images):
    # Prepare the URL
    keyword = urllib.parse.quote(keyword)
    url = f"https://www.google.com/search?q={keyword}&tbm=isch"

    # Send a GET request to the URL
    response = requests.get(url)

    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the image elements
    image_elements = soup.find_all("img", limit=num_images)

    # Extract the image URLs
    image_urls = [img["src"] for img in image_elements]

    return image_urls




In [11]:
# keyword to scraped
keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']

# Scrape images for each keyword
all_images = []
for keyword in keywords:
    images = scrape_images(keyword, 10)
    all_images.extend(images)

# Print the scraped image URLs
for i, image_url in enumerate(all_images, start=1):
    print(f"Image {i}: {image_url}")


Image 1: /images/branding/searchlogo/1x/googlelogo_desk_heirloom_color_150x55dp.gif
Image 2: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQF0IKCRPcaDP2Bcqb5K2xNM_5e3JkYdAQOJVXHSGgeoAECwFctx0jvjV14ub4&s
Image 3: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ86pCpQFnZQzJlSUCJ-i04qECrYBku6HdYZj-wObv4baRAki2NSYnimmzNpw&s
Image 4: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT9C-NJcANMo7LEwXJrm1_BL3pqX_dVakM44dDpY4oIKX0cDrLlEIfSfOLtmw&s
Image 5: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSAPRMJNugD2opEJjbKgi1_zT4e2KqeyOD0EkuhYBlGrFFUargNKmMz4dwL4n0&s
Image 6: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTTifgqC7e4QA3srbe0ZIbjQ7N52i8GGxDHtjtBNzjZoMlbk1PxY-uYTaAYMA&s
Image 7: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ8VLg_dIsviDrgnp4bTXbCTwNEYzRg9YbKyjDAoy97_coJkXFpqtcF7NBKWxU&s
Image 8: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQuAT50bfVOiskc4K5b4w3FkKq883yfA6tV9aixaUjm4wRpXjnr6pkv1nN5b9E&s
Image 9: https://encrypted-tbn

# Q4 Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com


In [102]:
# Initialize Chrome driver
driver1 = webdriver.Chrome(r"chromedriver.exe")  # Replace with the path to your chromedriver executable

# URL to scrape
url="https://www.flipkart.com/search?q=smartphone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"
driver1.get(url)

In [103]:
Brand_Name=[]
Colour=[]
Storage_RAM_ROM=[]
P_F_Camera=[]
Display_size_Resolution=[]
ProcessorAndCores=[]
Battery=[]
Price=[]
Product_URL=[]

In [104]:
#scraping the Brand_Name 
BName=driver1.find_elements(By.XPATH,"//div[@class='_4rR01T']")
for i in BName:
    if i.text is None :
        Brand_Name.append("--") 
    else:
        Brand_Name.append(i.text)
# print(len(Brand_Name),Brand_Name)

In [105]:
#scraping the Storage_RAM_ROM 
ram=driver1.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[1]")
for i in ram:
    if i.text is None :
        Storage_RAM_ROM.append("--") 
    else:
        Storage_RAM_ROM.append(i.text)
print(len(Storage_RAM_ROM),Storage_RAM_ROM)

24 ['4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '2 GB RAM | 32 GB ROM | Expandable Upto 512 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 512 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 2 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 2 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 2 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '3 GB RAM | 32 GB ROM | Expandable Upto 512 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 512 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '3 GB RAM | 32 GB ROM | Expandable Upto 512 GB', '8 GB RAM | 128 GB ROM', '3 GB RAM | 32 GB ROM | Expandable Upto 512 GB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '128 GB ROM', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '2 GB RAM | 32 GB ROM | Expandable Upto 512 GB', '4 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '4 GB RAM | 64 GB ROM | Expandable Upto 1 TB', '8 GB RAM | 128 GB ROM | Expandable Upto 1 TB', '

In [106]:
DS=driver1.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[2]")
for i in DS:
    if i.text is None :
        Display_size_Resolution.append("--") 
    else:
        Display_size_Resolution.append(i.text)
# print(len(Display_size_Resolution),Display_size_Resolution)

In [107]:
#scraping the P_F_Camera 
PC=driver1.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[3]")
for i in PC:
    if i.text is None :
        P_F_Camera.append("--") 
    else:
        P_F_Camera.append(i.text)
# print(len(P_F_Camera),P_F_Camera)

In [108]:
#scraping the ProcessorAndCores 
P=driver1.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[5]")
for i in P:
    if i.text is None :
        ProcessorAndCores.append("--") 
    else:
        ProcessorAndCores.append(i.text)
# print(len(ProcessorAndCores),ProcessorAndCores)

In [109]:
#scraping the Battery 
B=driver1.find_elements(By.XPATH,"//ul[@class='_1xgFaf']//li[4]")
for i in B:
    if i.text is None :
        Battery.append("--") 
    else:
        Battery.append(i.text)
# print(len(Battery),Battery)

In [110]:
#scraping the Price 
price=driver1.find_elements(By.XPATH,"//div[@class='_30jeq3 _1_WHN1']")
for i in price:
    if i.text is None :
        Price.append("--") 
    else:
        Price.append(i.text)
# print(len(Price),Price)

In [111]:
print(len(ProcessorAndCores))
#       ,len(Display_size_Resolution),len(ProcessorAndCores),len(Battery,Price)

24


In [112]:
df=pd.DataFrame([])
df['Brand_Name']=Brand_Name[:20]
df['Storage_RAM_ROM']=Storage_RAM_ROM[:20]
df['Amount P_F_Camera']=P_F_Camera[:20]
df['Display_size_Resolution']=Display_size_Resolution[:20]
df['ProcessorAndCores']=ProcessorAndCores[:20]
df['Battery']=Battery[:20]
df['Price']=Price[:20]


In [113]:
df

,Brand_Name,Storage_RAM_ROM,Amount P_F_Camera,Display_size_Resolution,ProcessorAndCores,Battery,Price
0,"SAMSUNG Galaxy F04 (Opal Green, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,13MP + 2MP | 5MP Front Camera,16.51 cm (6.5 inch) HD Display,Mediatek Helio P35 Processor,5000 mAh Lithium-Ion Battery,"₹8,499"
1,"POCO C50 (Royal Blue, 32 GB)",2 GB RAM | 32 GB ROM | Expandable Upto 512 GB,8MP Dual Camera | 5MP Front Camera,16.56 cm (6.52 inch) HD+ Display,"Mediatek Helio A22 Processor, Upto 2.0 GHz Pro...",5000 mAh Lithium Polymer Battery,"₹5,649"
2,"POCO M4 5G (Cool Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,50MP + 2MP | 8MP Front Camera,16.71 cm (6.58 inch) Full HD+ Display,Mediatek Dimensity 700 Processor,5000 mAh Lithium Ion Polymer Battery,"₹10,999"
3,"POCO C51 (Royal Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,8MP Dual Rear Camera | 5MP Front Camera,16.56 cm (6.52 inch) HD+ Display,Helio G36 Processor,5000 mAh Battery,"₹6,999"
4,"Infinix SMART 7 (Azure Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 2 TB,13MP + AI Lens | 5MP Front Camera,16.76 cm (6.6 inch) HD+ Display,Unisoc Spreadtrum SC9863A1 Processor,6000 mAh Battery,"₹7,299"
5,"Infinix SMART 7 (Night Black, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 2 TB,13MP + AI Lens | 5MP Front Camera,16.76 cm (6.6 inch) HD+ Display,Unisoc Spreadtrum SC9863A1 Processor,6000 mAh Battery,"₹7,299"
6,"Infinix SMART 7 (Emerald Green, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 2 TB,13MP + AI Lens | 5MP Front Camera,16.76 cm (6.6 inch) HD+ Display,Unisoc Spreadtrum SC9863A1 Processor,6000 mAh Battery,"₹7,299"
7,"POCO C55 (Cool Blue, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 1 TB,50MP Dual Rear Camera | 5MP Front Camera,17.04 cm (6.71 inch) HD+ Display,Mediatek Helio G85 Processor,5000 mAh Battery,"₹7,749"
8,"REDMI A1+ (Light Blue, 32 GB)",3 GB RAM | 32 GB ROM | Expandable Upto 512 GB,8MP Rear Camera | 5MP Front Camera,16.56 cm (6.52 inch) HD+ Display,Mediatek Helio A22 Processor,5000 mAh Lithium Polymer Battery,"₹6,599"
9,"POCO M4 5G (Power Black, 64 GB)",4 GB RAM | 64 GB ROM | Expandable Upto 512 GB,50MP + 2MP | 8MP Front Camera,16.71 cm (6.58 inch) Full HD+ Display,Mediatek Dimensity 700 Processor,5000 mAh Lithium Ion Polymer Battery,"₹10,999"


In [114]:
df.to_csv('Smartphone.csv')

In [115]:
driver1.close()

# Q5  Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps. 

In [97]:
driver = webdriver.Chrome(r"chromedriver.exe")

In [101]:
# opening google maps
driver.get("https://www.google.co.in/maps")
time.sleep(3)

city = input('Enter City Name : ')                                         # Enter city to be searched
search = driver.find_element(By.ID,"searchboxinput")                       # locating search bar
search.clear()                                                             # clearing search bar
time.sleep(2)
search.send_keys(city)                                                     # entering values in search bar
button = driver.find_element(By.ID,"searchbox-searchbutton")               # locating search button
button.click()                                                             # clicking search button
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City Name : bangalore
URL Extracted:  https://www.google.co.in/maps/place/Bengaluru,+Karnataka/@12.9539974,77.6309395,11z/data=!4m6!3m5!1s0x3bae1670c9b44e6d:0xf8dfc3e8517e4fe0!8m2!3d12.9715987!4d77.5945627!16zL20vMDljMTc?entry=ttu
Latitude = 12.9539974, Longitude = 77.6309395


# q6. Write a program to scrap all the available details of best gaming laptops from digit.in

In [66]:
# Specify the path to the WebDriver executable (e.g., chromedriver)
driver = webdriver.Chrome(r"chromedriver.exe")
# driver = webdriver.Chrome(driver_path)
driver.get("https://www.digit.in/top-products/best-gaming-laptops-40.html")

In [67]:

Brands=[]
Products_Description=[]
Specification=[]
Price=[]

In [78]:
br=driver.find_elements(By.XPATH,'//div[@class="TopNumbeHeading"]')
len(br)

7

In [79]:
for i in br:
   
    Brands.append(str(i.text).replace("\n",""))
Brands

['1.HP OMEN 17 (2023)',
 '2.MSI Titan GT77 12UHS',
 '3.Lenovo Legion 5i Pro',
 '4.ASUS ROG Strix Scar 18 2023',
 '5.Acer Predator Helios Neo 16',
 '6.ASUS ROG Zephyrus G14',
 '7.MSI Cyborg 15']

In [84]:
sp=driver.find_elements(By.XPATH,"//div[@class='Specs-Wrap']")
len(sp)

7

In [85]:
for i in sp:
   
    Specification.append(str(i.text).replace("\n",""))
# Specification

In [86]:

des=driver.find_elements(By.XPATH,"//div[@class='Section-center']")
len(des)

14

In [87]:
for i in des:
   
    Products_Description.append(str(i.text).replace("\n",""))

In [88]:
pri=driver.find_elements(By.XPATH,"//td[@class='smprice']")
len(pri)

7

In [89]:
for i in pri:
   
    Price.append(str(i.text).replace("\n",""))

In [90]:
digit=pd.DataFrame([])
digit['Brands']=Brands[0:7]
digit['Price']=Price[0:7]
digit['Specification']=Specification[0:7]
digit['Description']=Products_Description[0:7]
digit

,Brands,Price,Specification,Description
0,1.HP OMEN 17 (2023),"₹ 269,777","Windows 11 Home17.3"" (2560 x 1440)13th Gen Int...",We tested the HP OMEN 17 2023 at our digit tes...
1,2.MSI Titan GT77 12UHS,"₹ 499,990","Windows 11 Home17.3"" (3840 x 2160)12th Gen Int...",
2,3.Lenovo Legion 5i Pro,"₹ 213,900","Windows 11 Home16"" (2560 x 1600)12th Gen Intel...",This laptop won our best performance award las...
3,4.ASUS ROG Strix Scar 18 2023,"₹ 279,990","Windows 11 Home18"" (1920 x 1200)13th Gen Intel...",
4,5.Acer Predator Helios Neo 16,"₹ 149,990","Windows 11 Home16"" (2560 x 1600)13th Gen Intel...",This Lenovo Legion 5i Pro gaming laptop hits t...
5,6.ASUS ROG Zephyrus G14,"₹ 156,990","Windows 11 Home14"" (1920 x 1200)AMD Ryzen 9-69...",
6,7.MSI Cyborg 15,"₹ 130,990","Windows 11 Home15.6"" (1920 x 1080)12th Gen Int...",If a 17-inch display on a gaming laptop no lon...


In [102]:
driver.close()

# Q7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: 
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [101]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Send a GET request to the Forbes Billionaires page
url = "https://www.forbes.com/billionaires/"
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "html.parser")

# Find the table containing the billionaires' details
table = soup.find("table", class_="table")

if table is not None:
    # Find the table headers
    headers = table.find("thead").find_all("th")
    header_names = [header.text.strip() for header in headers]

    # Find the table rows
    rows = table.find("tbody").find_all("tr")

    # Initialize lists to store the details
    data = []
    for row in rows:
        # Find the cells within each row
        cells = row.find_all("td")

        # Extract the data from each cell
        row_data = [cell.text.strip() for cell in cells]

        # Append the row data to the main data list
        data.append(row_data)

    # Create a DataFrame from the collected details
    df = pd.DataFrame(data, columns=header_names)

    # Clean up the data
    df.rename(columns={"Net Worth": "Net worth", "Country/Territory": "Citizenship"}, inplace=True)
    df["Net worth"] = df["Net worth"].str.replace("[$€]", "").str.replace("B", "*1e9").str.replace("M", "*1e6").apply(pd.eval).astype(float)

    # Print the DataFrame
    print(df)
else:
    print("Table not found. Please check if the webpage structure has changed.")


Table not found. Please check if the webpage structure has changed.


In [77]:
# Create a DataFrame
e from the collected details
data = {
    "Rank": ranks,
    "Name": names,
    "Net Worth": net_worths,
    "Age": ages,
    "Citizenship": citizenships,
    "Source": sources,
    "Industry": industries
}
df = pd.DataFrame(data)

# Print the DataFrame
print(df)


Empty DataFrame
Columns: [Rank, Name, Net Worth, Age, Citizenship, Source, Industry]
Index: []


# Q8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video

In [63]:
# Set the path to your ChromeDriver executable
driver = webdriver.Chrome(r"chromedriver.exe")


driver.get("https://www.youtube.com/watch?v=YSWc2fGfTAs&ab_channel=StudyIQIAS")


In [64]:

# Scroll down to load more comments
SCROLL_PAUSE_TIME = 1
last_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Find comment elements
comments = driver.find_elements(By.CSS_SELECTOR, "#comment #content-text")
timestamps = driver.find_elements(By.CSS_SELECTOR, "#header-author > yt-formatted-string > a")

# Extract comment text and timestamps
comment_list = [comment.text for comment in comments]
timestamp_list = [timestamp.get_attribute("innerHTML") for timestamp in timestamps]


In [65]:
df = pd.DataFrame({"Comment": comment_list, "Timestamp":timestamp_list})

# Print the DataFrame
print(df)

                                               Comment     Timestamp
0    UPSC IAS (Pre + Mains) LIVE Foundation 2024 Ba...  4 months ago
1    Hats off to you sir for providing such great q...  4 months ago
2    Was waiting for your comprehensive yet crisp c...  4 months ago
3    You have explained very briefly. Even a studen...  4 months ago
4    Thanks for every efforts you are putting in fo...  4 months ago
..                                                 ...           ...
478                        Tomato onion potato\n        4 months ago
479  One of the Worst budget in Independent India's...  4 months ago
480  NOW INDIA 🇮🇳 IS GROWING IN WORLD  FASTEST ECON...  4 months ago
481                            Adani chhor hai\n        4 months ago
482                                          0\n        4 months ago

[483 rows x 2 columns]


# Q9 Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 


In [19]:
driver = webdriver.Chrome(r"chromedriver.exe")


driver.get("https://www.hostelworld.com/search?city=London&country=England")


In [58]:
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.documentElement.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Find all hostel containers
hostel_containers = driver.find_elements(By.CSS_SELECTOR, ".property-card")

# Initialize lists to store the data
hostel_names = []
distances = []
ratings = []
total_reviews = []
overall_reviews = []
privates_prices = []
dorms_prices = []
facilities = []
descriptions = []

# Iterate over each hostel container
for container in hostel_containers:
    # Hostel name
    hostel_name = container.find_element(By.CSS_SELECTOR, ".title").text.strip()
    hostel_names.append(hostel_name)

    dist = container.find_element(By.XPATH, "//span[@class='description']").text.strip()
    distances.append(dist)

    # Ratings
    rating = container.find_element(By.XPATH, "//div[@class='score orange big']").text.strip()
    ratings.append(rating)

    # Total reviews
    total_review = container.find_element(By.CSS_SELECTOR, ".reviews").text.strip()
    total_reviews.append(total_review)

    # Overall reviews
    overall_review = container.find_element(By.CSS_SELECTOR, ".keyword").text.strip()
    overall_reviews.append(overall_review)

    # Privates prices
    privates_price = container.find_element(By.CSS_SELECTOR, ".price").text.strip()
    privates_prices.append(privates_price)


    # Facilities
    facility = container.find_element(By.XPATH, "//div[@class='facilities-label']").text.strip()
    facilities.append(facility)


    # Property description
    des = container.find_element(By.XPATH, "//p[@class='intro-text']").text.strip()
    descriptions.append(des)


In [59]:
# Create DataFrame
data = {
    "Hostel Name": hostel_names,
    "Distance from City Centre": distances,
    "Rating": ratings,
    "Total Reviews": total_reviews,
    "Overall Reviews": overall_reviews,
    "Privates from Price": privates_prices,   
    "Facilities": facilities,
   "Property Description": descriptions
}
df = pd.DataFrame(data)

In [60]:
df

,Hostel Name,Distance from City Centre,Rating,Total Reviews,Overall Reviews,Privates from Price,Facilities,Property Description
0,St Christopher's Village,Hostel - 5.47km from city centre,9.6,12143,Very Good,Rs2390,Free WiFi,The perfect place for solo travelers to connec...
1,Onefam Notting Hill by Hostel One,Hostel - 5.47km from city centre,9.6,2049 Total Reviews,Superb,Rs3299,Free WiFi,The perfect place for solo travelers to connec...
2,Urbany Hostel London,Hostel - 5.47km from city centre,9.6,775 Total Reviews,Superb,Rs4084,Free WiFi,The perfect place for solo travelers to connec...
3,YHA London St Pancras,Hostel - 5.47km from city centre,9.6,2868 Total Reviews,Superb,Rs3314,Free WiFi,The perfect place for solo travelers to connec...
4,Destinations Hostels @ The Gallery,Hostel - 5.47km from city centre,9.6,159 Total Reviews,Superb,Rs3107,Free WiFi,The perfect place for solo travelers to connec...
5,International Students House,Hostel - 5.47km from city centre,9.6,1034 Total Reviews,Superb,Rs6265,Free WiFi,The perfect place for solo travelers to connec...
6,Selina Camden,Hostel - 5.47km from city centre,9.6,49 Total Reviews,Superb,Rs5219,Free WiFi,The perfect place for solo travelers to connec...
7,Barmy Badger Backpackers,Hostel - 5.47km from city centre,9.6,1922 Total Reviews,Superb,Rs2900,Free WiFi,The perfect place for solo travelers to connec...
8,YHA London Earls Court,Hostel - 5.47km from city centre,9.6,1459 Total Reviews,Superb,Rs1726,Free WiFi,The perfect place for solo travelers to connec...
9,Wombat's City Hostel London,Hostel - 5.47km from city centre,9.6,14875 Total Reviews,Superb,Rs3573,Free WiFi,The perfect place for solo travelers to connec...
